In [2]:
address = '4047 Bremo Recess, New Albany, OH 43054'
street = address.split(',')[0]
city = address.split(',')[1].strip()
state = address.split(',')[2].split()
state = ''.join(state)[0:2]
zipcode = address.split(',')[2].split()[1]

In [3]:
!pip install folium

In [4]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as color
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup
import regex as re

In [5]:
geolocator = Nominatim(user_agent = 'calc')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [6]:
tooltip_1 = address
map1 = folium.Map(location=[latitude, longitude], tiles='openstreetmap', zoom_start=15)
folium.Marker(
    [latitude, longitude], popup="Victoria Memorial", tooltip=tooltip_1).add_to(map1)

In [7]:
map1

In [8]:
folium.Circle(
    location=[latitude, longitude],
    radius=1000,
    popup="Salt Lake",
    color="blue",
    fill=True,
).add_to(map1)

In [9]:
map1

In [10]:
CLIENT_ID = 'RN530BC0LUAPUIO15YIWGNE0NKN3OWUM4CE4EF5YAL1PKGK2' # your Foursquare ID
CLIENT_SECRET = 'PEFFPSDRAKNQNDMRXK5HGWUHVDFEAWTL5P4AOM11SZTMVMOO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RN530BC0LUAPUIO15YIWGNE0NKN3OWUM4CE4EF5YAL1PKGK2
CLIENT_SECRET:PEFFPSDRAKNQNDMRXK5HGWUHVDFEAWTL5P4AOM11SZTMVMOO


In [11]:
radius = 1000
LIMIT = 100
url = 'http://<analysis url>/FindHotSpots/submitJob'

results = requests.get(url).json()
results

ConnectionError: HTTPConnectionPool(host='%3canalysis%20url%3e', port=80): Max retries exceeded with url: /FindHotSpots/submitJob (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001BB9F169820>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [128]:
venues = results['response']['groups'][0]['items']
    
venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.location.city', 'venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venues = venues.loc[:, filtered_columns]

# filter the category for each row
venues['venue.categories'] = venues.apply(get_category_type, axis=1)

# clean columns
venues.columns = [col.split(".")[-1] for col in venues.columns]

venues

,city,name,categories,lat,lng
0,New Albany,Oakland Nursery,Garden Center,40.061848,-82.832065
1,Gahanna,The Barn at Rocky Fork,Steakhouse,40.053262,-82.842731
2,Gahanna,1837 Wine & Spirits Emporium,Liquor Store,40.053718,-82.843663
3,Gahanna,Donatos Pizza,Pizza Place,40.054123,-82.843783
4,NaN,inside and out,Garden Center,40.062131,-82.831267
5,Gahanna,Harrison Pond Park Gahanna,Park,40.052212,-82.838474
6,Gahanna,Morse Rd & Johnstown Rd Roundabout,Intersection,40.054561,-82.841823
7,Gahanna,Genesis Audio Ltd.,Electronics Store,40.053917,-82.843237
8,Gahanna,Salon Therapy,Cosmetics Shop,40.053979,-82.843359
9,NaN,Head Of Pond Rd.,Trail,40.057464,-82.823275


In [124]:
venues = venues.rename(columns = {'city': 'City', 'name': 'Name', 'categories': 'Categories', 'lat': 'Latitude', 'lng': 'Longitude'})

In [125]:
print('There are {} venues near the address that were returned by Foursquare.'.format(venues.shape[0]))

There are 10 venues near the address that were returned by Foursquare.


In [126]:
print(venues.shape)
venues.head()

(10, 5)


,City,Name,Categories,Latitude,Longitude
0,New Albany,Oakland Nursery,Garden Center,40.061848,-82.832065
1,Gahanna,The Barn at Rocky Fork,Steakhouse,40.053262,-82.842731
2,Gahanna,1837 Wine & Spirits Emporium,Liquor Store,40.053718,-82.843663
3,Gahanna,Donatos Pizza,Pizza Place,40.054123,-82.843783
4,NaN,inside and out,Garden Center,40.062131,-82.831267
